In [1]:
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np

## Training and validation dataset generation

In [2]:
train=tf.keras.utils.image_dataset_from_directory(
    'Dataset',
    validation_split=0.2,
    image_size=(256,256),
    subset='training',
    seed=123,
)
validation=tf.keras.utils.image_dataset_from_directory(
    'Dataset',
    image_size=(256,256),
    validation_split=0.2,
    subset='validation',
    seed=123,
)

AUTOTUNE=tf.data.AUTOTUNE
train=train.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation=validation.cache().prefetch(buffer_size=AUTOTUNE)

Found 7006 files belonging to 3 classes.
Using 5605 files for training.
Found 7006 files belonging to 3 classes.
Using 1401 files for validation.


In [3]:
augmentation_layers= tf.keras.Sequential(
    [
    tf.keras.layers.Resizing(256,256),
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.15),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomShear(0.1),
    ]  
)

## XGBoost with VGG classifier


VGG

In [60]:
from keras.applications.vgg16 import VGG16

VGG_model=VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(256,256,3)
)
for layer in VGG_model.layers:
    layer.trainable = False
VGG_model.summary()


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_11 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 256, 256, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 256, 256, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 128, 128, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 64, 64, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 64, 64, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 64, 64, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 32, 32, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 16, 16, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 8, 8, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

Our dataset is organized in batches so we must process each batch.

In [62]:
import numpy as np

def extract_features(dataset, model, augment):
    features_list = []
    labels_list = []
    for batch_images, batch_labels in dataset:
        batch_images= augment(batch_images)
        batch_images= batch_images / 255.0
        features= model.predict(batch_images)
        features= features.reshape(features.shape[0], -1)
        features_list.append(features)
        labels_list.append(batch_labels.numpy())
    X = np.vstack(features_list)
    y = np.concatenate(labels_list).ravel()
    return X, y

X_train,y_train = extract_features(train, VGG_model, augment=augmentation_layers)
X_val,y_val = extract_features(validation, VGG_model, augment=augmentation_layers)

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 866ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
print(class_weights)

[2.08985831 2.10872837 0.48845316]


In [ ]:
import xgboost as xgb
from sklearn.metrics import classification_report
sample_weights = class_weights[y_train]
xgboost = xgb.XGBClassifier(
)
xgboost.fit(X_train, y_train, sample_weight=sample_weights)
y_pred = xgboost.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.56      0.62       199
           1       0.54      0.33      0.41       244
           2       0.81      0.92      0.86       958

    accuracy                           0.77      1401
   macro avg       0.68      0.61      0.63      1401
weighted avg       0.75      0.77      0.75      1401



## ResNet

In [19]:
pretrained_model=tf.keras.applications.resnet50.ResNet50(
    weights='imagenet',
    include_top=False,
    pooling='avg',
    classes=3,
)

In [20]:
for layer in pretrained_model.layers:
    layer.trainable = False
resnet_model = tf.keras.Sequential()
resnet_model.add(pretrained_model)
resnet_model.add(tf.keras.layers.BatchNormalization())
resnet_model.add(tf.keras.layers.Dropout(0.3))
resnet_model.add(tf.keras.layers.Flatten())
resnet_model.add(tf.keras.layers.Dense(1024, activation="relu"))
resnet_model.add(tf.keras.layers.Dropout(0.3))
resnet_model.add(tf.keras.layers.Dense(512, activation="relu"))
resnet_model.add(tf.keras.layers.Dropout(0.3))
resnet_model.add(tf.keras.layers.Dense(256, activation="relu"))
resnet_model.add(tf.keras.layers.Dropout(0.3))
resnet_model.add(tf.keras.layers.Dense(3, activation="softmax"))
resnet_model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 2048)           │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 2048)           │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,350,979 (100.52 MB)

 Trainable params: 2,759,171 (10.53 MB)

 Non-trainable params: 23,591,808 (90.00 MB)

In [14]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

def load_data(dataset):
    features_list = []
    labels_list = []
    for batch_images, batch_labels in dataset:
        batch_images= batch_images / 255.0
        features_list.append(batch_images.numpy())
        labels_list.append(batch_labels.numpy())
    X = np.vstack(features_list)
    y = np.concatenate(labels_list).ravel()
    return X, y
X_train, y_train = load_data(train)
X_val, y_val = load_data(validation)
class_weight = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight = dict(enumerate(class_weight))


In [16]:
class_weight

{0: np.float64(2.0898583146905296),
 1: np.float64(2.1087283671933785),
 2: np.float64(0.48845315904139436)}

In [22]:
adagrad=tf.keras.optimizers.Adagrad()
resnet_model.compile(
    optimizer=adagrad, 
    loss='sparse_categorical_crossentropy', #y_train is not one-hot encoded, y_train is in integer format
    metrics=['accuracy'])
resnet_model.fit(train,
validation_data=validation, 
epochs=3,
class_weight=class_weight) 
resnet_model.save('resnet50_model_with_da.keras')

Epoch 1/3
176/176 ━━━━━━━━━━━━━━━━━━━━ 391s 2s/step - accuracy: 0.4303 - loss: 1.1987 - val_accuracy: 0.6060 - val_loss: 0.8744
Epoch 2/3
176/176 ━━━━━━━━━━━━━━━━━━━━ 384s 2s/step - accuracy: 0.5236 - loss: 1.0321 - val_accuracy: 0.6124 - val_loss: 0.8659
Epoch 3/3
176/176 ━━━━━━━━━━━━━━━━━━━━ 380s 2s/step - accuracy: 0.5707 - loss: 0.9519 - val_accuracy: 0.6395 - val_loss: 0.8280


## AlexNet with SVM classifier